# Bank Marketing - Exploration des données

**Objectif** : Prédire si un client souscrira à un dépôt à terme suite à une campagne de télémarketing.

**Source** : UCI Bank Marketing Dataset (institution bancaire portugaise, 2008-2010)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
%matplotlib inline

## 1. Chargement des données

In [ ]:
# Le fichier utilise ';' comme séparateur
df = pd.read_csv('data/bank-full.csv', sep=';')
print(f"Dimensions : {df.shape[0]} lignes, {df.shape[1]} colonnes")

In [ ]:
df.head()

## 2. Aperçu des données

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# Vérification des valeurs manquantes
print("Valeurs manquantes par colonne :")
print(df.isnull().sum())

## 3. Variable cible (y)

In [ ]:
# Distribution de la variable cible
target_counts = df['y'].value_counts()
target_pct = df['y'].value_counts(normalize=True) * 100

print("Distribution de la variable cible :")
print(target_counts)
print(f"\nPourcentages : no={target_pct['no']:.1f}%, yes={target_pct['yes']:.1f}%")

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
colors = ['#e74c3c', '#2ecc71']
df['y'].value_counts().plot(kind='bar', color=colors, ax=ax)
ax.set_title('Distribution de la variable cible')
ax.set_xlabel('Souscription au dépôt à terme')
ax.set_ylabel('Nombre de clients')
ax.set_xticklabels(['Non', 'Oui'], rotation=0)
plt.tight_layout()
plt.show()

## 4. Variables catégorielles

In [ ]:
cat_cols = df.select_dtypes(include='object').columns.tolist()
print(f"Variables catégorielles ({len(cat_cols)}) : {cat_cols}")

In [ ]:
# Valeurs uniques par variable catégorielle
for col in cat_cols:
    print(f"\n{col} ({df[col].nunique()} valeurs) :")
    print(df[col].value_counts())

## 5. Variables numériques

In [ ]:
num_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
print(f"Variables numériques ({len(num_cols)}) : {num_cols}")

In [ ]:
# Distribution des variables numériques
fig, axes = plt.subplots(3, 3, figsize=(12, 10))
axes = axes.flatten()

for i, col in enumerate(num_cols):
    if i < len(axes):
        axes[i].hist(df[col], bins=30, edgecolor='black', alpha=0.7)
        axes[i].set_title(col)
        axes[i].set_xlabel('')

# Masquer les axes inutilisés
for j in range(len(num_cols), len(axes)):
    axes[j].set_visible(False)

plt.tight_layout()
plt.show()

## 6. Relation entre features et variable cible

In [ ]:
# Taux de souscription par catégorie de job
job_rates = df.groupby('job')['y'].apply(lambda x: (x == 'yes').mean() * 100).sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(10, 5))
job_rates.plot(kind='bar', ax=ax, color='steelblue')
ax.set_title('Taux de souscription par type de job')
ax.set_xlabel('Job')
ax.set_ylabel('Taux de souscription (%)')
ax.axhline(y=target_pct['yes'], color='red', linestyle='--', label=f'Moyenne ({target_pct["yes"]:.1f}%)')
ax.legend()
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Distribution de l'âge selon la souscription
fig, ax = plt.subplots(figsize=(8, 5))
df[df['y'] == 'no']['age'].hist(bins=30, alpha=0.5, label='Non', ax=ax)
df[df['y'] == 'yes']['age'].hist(bins=30, alpha=0.5, label='Oui', ax=ax)
ax.set_title("Distribution de l'âge selon la souscription")
ax.set_xlabel('Âge')
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Durée du dernier contact vs souscription
fig, ax = plt.subplots(figsize=(8, 5))
df.boxplot(column='duration', by='y', ax=ax)
ax.set_title('Durée du dernier contact selon la souscription')
ax.set_xlabel('Souscription')
ax.set_ylabel('Durée (secondes)')
plt.suptitle('')  # Supprimer le titre automatique
plt.tight_layout()
plt.show()

## 7. Observations préliminaires

**À compléter après exécution du notebook :**

- Classes déséquilibrées : ~88% no vs ~12% yes
- Variables catégorielles à encoder : job, marital, education, default, housing, loan, contact, month, poutcome
- Variable `duration` potentiellement très prédictive (mais attention : non disponible avant l'appel)
- Valeurs `unknown` présentes dans certaines colonnes catégorielles
- `pdays = -1` signifie que le client n'a pas été contacté lors de campagnes précédentes